# Proyecto 1 - NLP

## Revision de ___ en comentarios de libros 

#### Importación de librerias

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import re

from wordcloud import WordCloud, STOPWORDS

#Librerias para sampling
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

#Librerias para vectorizacion
from sklearn.feature_extraction.text import TfidfVectorizer

#### Importación de datos

In [ ]:
data = pd. read_csv('data/reviews.csv')

#### Preparación e los datos

1. Se se paro la columna helpfull en dpos columnas. Las veces que se le dio helpfull postivo, y las veces totales que se le ha dado helpful

In [ ]:
data['Helpful Total'] = data['helpful'].str.replace("[","")
data['Helpful Total'] = data['Helpful Total'].str.replace("]","")
separacion = data['Helpful Total'].str.split(",",expand=True)
data['Helpful Total']=separacion[1].astype(int)
data['Helpful Positivo']=separacion[0].astype(int)
data

2. Se omiteron los regitso que no han sido calificados helpfull ninguna vezy se calculo la tasa de positividad

In [ ]:
# Filtar los reviws que no han sido clasificados como helpful
data = data[data["Helpful Total"] >0 ]
data['Helpful Rate'] = data['Helpful Positivo']/data['Helpful Total']
data.head()

#### Análisis de los datos

 Distribución de la tasa de postividad en helpfull

In [ ]:
sns.displot(data["Helpful Rate"],bins = 10).set(title="Distribución del helpful rate");

#### Definición  de la vriable de salida

In [ ]:
data["Output"] = np.where(data["Helpful Rate"] >= 0.9,1, 0)
sns.countplot(x="Output", data=data)

#### Dataframe NLP 

In [ ]:
df = data[["Output", "reviewText"]]
df = df.reset_index()
df = df.drop(columns=["index"])
df.head()

#### Procesamiento de datos

In [ ]:
import string
import emoji
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import en_core_web_sm
import unidecode
import unicodedata

In [ ]:
# Puntucacion
punctuations = string.punctuation
# Carga los modelos de spacy y las stopwords
parser = en_core_web_sm.load()
stop_words = STOP_WORDS

In [ ]:
def clean_text(text):
    # Elimina espacios, emojis, números y convierte todo a minúscula.
    text = str(text)
    text = text.replace('\n','').replace('\r','').replace('\t','').strip().lower()
    text = re.sub(r'\d+', '', text)
    allchars = [str for str in text] 
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    text=''.join((c for c in unicodedata.normalize('NFD',text) if unicodedata.category(c) != 'Mn'))
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    # Tokenize
    token = parser(text)
    # Lemmatize
    token = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in token]
    # Quita stopwords y signos de puntuación
    token = ' '.join([word for word in token if word not in stop_words and word not in punctuations])
    token = unidecode.unidecode(token)
    token = ' '.join([w for w in token.split() if len(w)>1])
    return token

In [ ]:
df["CleanText"]=df["reviewText"].apply(clean_text)
df.head(10)

In [ ]:
df =df.drop(columns="reviewText")
df.to_csv("dataNLP.csv")

#### Exploración

In [ ]:
plt.figure(figsize=(14,7))
positive_text = ''.join(df[df['Output'] == 1].CleanText)
word_cloud = WordCloud(stopwords = STOPWORDS, max_words = 100, width=1366, height=768, background_color="white").generate(positive_text)
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word cloud de comentarios postivos',fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(14,7))
negative_text = ''.join(str(df[df['Output'] == 0].CleanText))
word_cloud = WordCloud(stopwords = STOPWORDS, max_words = 100, width=1366, height=768, background_color="white").generate(negative_text)
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word cloud de comentarios negativos',fontsize=20)
plt.show()

#### Conversión a vectores

Importación del nuevo dataset. Solo se toman  50000 registros porque si no es muy pesado para crear lasm atrices y ajustar el modelo.  

In [3]:
df =pd.read_csv('dataNLP.csv', index_col=0)
df=df.dropna()
df=df.iloc[0:50000]


Se separa la muestra 90% train y 10% test

In [7]:
Y = df["Output"]
X = df["CleanText"]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state = 42)

Se codifican las matrices bajo el algoritmo TF_IDF. Solo se incluyen las ppalabras que están más de 50 veces en el dataframe para evitar palabras no significativas. 

In [8]:
tfidf_vectorizer = TfidfVectorizer(min_df = 50)

x_train_vect = tfidf_vectorizer.fit_transform(x_train).toarray()
x_test_vect = tfidf_vectorizer.transform(x_test).toarray()

In [9]:
x_train_vect.shape

(45000, 4280)

#### Modelo Random Forest con represntación TF - IDF 

Se crea un Pipeline con un modelo RanfomForest que se calibra por medio de  RandomSearchCV para encontrar los hiperparámetros que minimizan el error.

In [10]:
Pipeline_RF = Pipeline(steps = [("RF", RandomForestClassifier())])

Param_RF = {
    'RF__max_depth': np.linspace(1, 20, 20, endpoint=True),
    'RF__max_features': ['auto','log2','sqrt',None],
    'RF__min_samples_split':np.linspace(0.1, 1.0, 10, endpoint=True),
    'RF__min_samples_leaf':np.linspace(0.1, 0.5, 5, endpoint=True)  
}

Grid_RF = RandomizedSearchCV(estimator = Pipeline_RF, param_distributions = Param_RF, scoring = 'accuracy', cv = 5, n_iter = 100, n_jobs = -1)

In [11]:
Grid_RF.fit(x_train_vect,y_train)
print('Mejores Parametros: %s' % Grid_RF.best_params_)
print('Mejor AUC en Train: %.3f' % Grid_RF.best_score_)

Mejores Parametros: {'RF__min_samples_split': 0.2, 'RF__min_samples_leaf': 0.5, 'RF__max_features': None, 'RF__max_depth': 16.0}
Mejor AUC en Train: 0.592


##### Resulatados del modelo 

In [13]:
y_pred = Grid_RF.best_estimator_.predict(x_test_vect)
y_pred_proba = Grid_RF.best_estimator_.predict_proba(x_test_vect)[:,1]
print('La precisión del modelo Random Forest con TF- IDF  calibrado es:', '{:.2f}'.format(accuracy_score(y_test, y_pred)))
print('El AUC del modelo Random Forest con TF- IDF calibrado es:', '{:.2f}'.format(roc_auc_score(y_test, y_pred_proba)))

La precisión del modelo Random Forest con TF- IDF  calibrado es: 0.59
El AUC del modelo Random Forest con TF- IDF calibrado es: 0.50


####  Modelo 2

####  Modelo 3